In [ ]:
import os
import dotenv
dotenv.load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "SQL test"

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langgraph.prebuilt import ToolNode
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

from typing import Dict, List, Any

# Load the database
db = SQLDatabase.from_uri("sqlite:///data.db")

# Initialize the LLM and toolkit
llm = ChatOpenAI(model="gpt-4o-mini")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

# Tools setup
list_tables_tool = next(tool for tool in tools if tool.name == "sql_db_list_tables")
get_schema_tool = next(tool for tool in tools if tool.name == "sql_db_schema")

# Tool for handling SQL queries
@tool
def db_query_tool(query: str) -> str:
    """
    Execute a SQL query against the database and get back the result.
    If the query is not correct, an error message will be returned.
    """
    result = db.run_no_throw(query)
    if not result:
        return "Error: Query failed. Please rewrite your query and try again."
    return result

# New Tool for appointment booking
@tool
def appointment_booking_tool(action: str) -> str:
    """
    Responds to user inquiries about booking a test drive or buying a car.
    """
    return "You can visit our dealership anytime from 8am to 10pm, Monday to Friday, to book an appointment or take a test drive."

# Function to create a ToolNode with fallback
def create_tool_node_with_fallback(tools: list) -> RunnableWithFallbacks[Any, dict]:
    """
    Create a ToolNode with a fallback to handle errors and surface them to the agent.
    """
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )

def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }

# Chat prompt template for query checking
query_check_system = """You are a SQL expert with a strong attention to detail.
Double check the SQLite query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

You will call the appropriate tool to execute the query after running this check."""

query_check_prompt = ChatPromptTemplate.from_messages(
    [("system", query_check_system), ("placeholder", "{messages}")]
)

query_check = query_check_prompt | llm.bind_tools(
    [db_query_tool], tool_choice="required"
)

# Use langgraph to build a ToolNode that can handle diverse inputs
tool_node = ToolNode([
    db_query_tool,
    appointment_booking_tool,
    list_tables_tool,
    get_schema_tool,
]).with_fallbacks([RunnableLambda(handle_tool_error)])

# Handle user messages with langgraph's ToolNode
def handle_user_message(message: str) -> str:
    """
    Handles the user message by using langgraph to detect the appropriate tool or respond directly.
    """
    # Process the message through the tool node
    result = tool_node.invoke({"input": message})
    
    # If no tool was invoked, return a generic response
    if not result:
        return f"Hello, I'm Dumbledor from the dealership. How can I assist you today?"
    
    return result["output"]

# Example usage:
response = handle_user_message("I would like to book a test drive.")
print(response)


In [ ]:
response = handle_user_message("Hello")
print(response)

In [ ]:

response = handle_user_message("how many kias do you have?")
print(response)